# DNA Methylation Analysis Tutorial

This notebook provides a step-by-step guide to running the DNA methylation analysis pipeline for male fertility classification.

## Overview

The analysis includes:
1. **Data Loading** - Load methylation data and metadata
2. **Feature Selection** - Select probes with high blood-sperm correlation
3. **Model Training** - Train classification models (Logistic Regression, Naive Bayes, PLS-DA)
4. **Validation** - Internal (Blood→Sperm) and external (GSE51245) validation
5. **Visualization** - ROC curves, PCA plots, correlation histograms

## Prerequisites

Make sure you have installed all dependencies:
```bash
pip install -r requirements.txt
```

## 1. Setup and Imports

In [ ]:
# Standard library imports
import sys
from pathlib import Path

# Add project root to path
PROJECT_ROOT = Path.cwd().parent
sys.path.insert(0, str(PROJECT_ROOT))

# Scientific computing
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# ML libraries
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Project modules
from src.utils.config import load_config
from src.data_loaders import MethylationDataLoader, MetadataLoader
from src.preprocessing import CorrelationSelector, DataTransformer
from src.models import ClassifierFactory, ClassifierEvaluator
from src.visualization import PlotGenerator, setup_publication_style

# Set up matplotlib for inline plotting
%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['figure.dpi'] = 100

print("All imports successful!")
print(f"Project root: {PROJECT_ROOT}")

## 2. Configuration

The configuration system allows you to easily switch between different datasets and probe sets.

In [ ]:
# Load configuration
# You can change these parameters to analyze different datasets/probe sets

DATASET = "GSE213366"  # Options: "GSE213366", "GSE51245", "GSE149318"
PROBE_SET = "CoRSIV"   # Options: "CoRSIV", "Controls", "EPIC_full"

config = load_config(dataset=DATASET, probe_set=PROBE_SET)

print(f"Configuration loaded:")
print(f"  Dataset: {config.dataset_name}")
print(f"  Probe set: {config.probe_set_name}")
print(f"  Base directory: {config.base_dir}")

In [ ]:
# View available parameters
print("Model parameters:")
for key, value in config.model_params.items():
    print(f"  {key}: {value}")

print("\nClass mapping:")
print(f"  {config.class_mapping}")

## 3. Load Data

Load the methylation data and metadata using the data loaders.

In [ ]:
# Initialize data loaders
meth_loader = MethylationDataLoader(config)
meta_loader = MetadataLoader(config)

# Define file paths
# Using classifier data for CoRSIV probes
METH_FILE = PROJECT_ROOT / "classifier" / "data" / "GSE213366_CoRSIV_2023.csv"
META_FILE = PROJECT_ROOT / "classifier" / "data" / "sample_tissue_individual_map.csv"

print(f"Loading methylation data from: {METH_FILE.name}")
print(f"Loading metadata from: {META_FILE.name}")

In [ ]:
# Load methylation data
meth_df = meth_loader.load(METH_FILE)

print(f"\nMethylation data shape: {meth_df.shape}")
print(f"  Samples: {meth_df.shape[0]}")
print(f"  Probes: {meth_df.shape[1]}")
meth_df.head()

In [ ]:
# Load metadata
meta_df = meta_loader.load(META_FILE)

print(f"\nMetadata shape: {meta_df.shape}")
print(f"\nColumns: {meta_df.columns.tolist()}")
meta_df.head()

In [ ]:
# Explore the data distribution
print("Sample distribution:")
print(f"\nBy tissue:")
print(meta_df['tissue'].value_counts())

print(f"\nBy status:")
print(meta_df['status'].value_counts())

print(f"\nCrosstab (tissue x status):")
print(pd.crosstab(meta_df['tissue'], meta_df['status']))

## 4. Merge Data and Prepare for Analysis

In [ ]:
# Merge methylation data with metadata
merged_df = meta_df.merge(meth_df, left_on='sample_name', right_index=True)

# Encode target variable
merged_df['target'] = merged_df['status'].map({
    'Normozoospermia': 0,
    'Oligozoospermia': 1
})

print(f"Merged data shape: {merged_df.shape}")
merged_df[['sample_name', 'tissue', 'status', 'target']].head()

## 5. Feature Selection

Select probes with high blood-sperm correlation to ensure biomarkers are transferable across tissues.

In [ ]:
# Load pre-computed correlation data
CORR_FILE = PROJECT_ROOT / "classifier" / "data" / "blood_sperm_correlation_results.csv"

if CORR_FILE.exists():
    corr_df = pd.read_csv(CORR_FILE)
    print(f"Loaded correlation data for {len(corr_df)} probes")
    
    # Visualize correlation distribution
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.hist(corr_df['Pearson_Correlation'].dropna(), bins=50, color='steelblue', alpha=0.7)
    ax.axvline(0.75, color='red', linestyle='--', label='Threshold (0.75)')
    ax.set_xlabel('Pearson Correlation (r)')
    ax.set_ylabel('Number of Probes')
    ax.set_title('Distribution of Blood-Sperm Probe Correlations')
    ax.legend()
    plt.show()
    
    # Count probes above threshold
    n_above = (corr_df['Pearson_Correlation'] > 0.75).sum()
    print(f"\nProbes with r > 0.75: {n_above}")
else:
    print("Correlation file not found. Will use all probes.")

In [ ]:
# Select features using CorrelationSelector
selector = CorrelationSelector(threshold=0.75)

if CORR_FILE.exists():
    selector.fit_from_file(str(CORR_FILE))
    selected_features = selector.get_selected_features()
    
    # Filter to features available in our data
    meta_cols = ['Individual_ID', 'geo_accession', 'sample_name', 'tissue', 'status', 'target']
    probe_cols = [c for c in merged_df.columns if c not in meta_cols]
    
    available_features = [f for f in selected_features if f in probe_cols]
    print(f"Selected {len(available_features)} features for classification")
else:
    available_features = probe_cols
    print(f"Using all {len(available_features)} probes")

## 6. Split Data by Tissue

Split data into training (Blood) and test (Sperm) sets.

In [ ]:
# Split by tissue
train_df = merged_df[merged_df['tissue'] == 'Blood'].copy()
test_df = merged_df[merged_df['tissue'] == 'Sperm'].copy()

print(f"Training set (Blood): {len(train_df)} samples")
print(f"Test set (Sperm): {len(test_df)} samples")

# Prepare features and targets
X_train = train_df[available_features]
y_train = train_df['target']

X_test = test_df[available_features]
y_test = test_df['target']

# Handle missing values
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_train.mean())  # Use training means

print(f"\nFeature matrix shape: {X_train.shape}")

## 7. Train Classification Models

In [ ]:
# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Features scaled using StandardScaler")

In [ ]:
# Create and train models
factory = ClassifierFactory(config)
evaluator = ClassifierEvaluator(cv_folds=5, random_state=42)

# Get all models
models = factory.create_all()
print(f"Created models: {list(models.keys())}")

# Evaluate all models
results = evaluator.evaluate_all(
    models, 
    X_train_scaled, y_train.values,
    X_test_scaled, y_test.values
)

In [ ]:
# Display results
results_df = evaluator.to_dataframe(results)
results_df

## 8. Visualize Results

In [ ]:
# Plot ROC curves
from sklearn.metrics import roc_curve

fig, ax = plt.subplots(figsize=(8, 6))

colors = ['#2ecc71', '#3498db', '#e74c3c']

for (name, metrics), color in zip(results.items(), colors):
    if not np.isnan(metrics['auc']):
        fpr, tpr, _ = roc_curve(y_test, metrics['y_prob'])
        ax.plot(fpr, tpr, lw=2, color=color, 
                label=f"{name} (AUC = {metrics['auc']:.3f})")

ax.plot([0, 1], [0, 1], 'k--', lw=1.5, alpha=0.5)
ax.set_xlim([0.0, 1.0])
ax.set_ylim([0.0, 1.05])
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.set_title('ROC Curves: Classification of Oligozoospermia\n(Training: Blood, Test: Sperm)')
ax.legend(loc='lower right')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# PCA visualization
# Combine train and test for visualization
X_combined = np.vstack([X_train_scaled, X_test_scaled])
y_combined = np.concatenate([y_train.values, y_test.values])
tissue_labels = ['Blood'] * len(X_train) + ['Sperm'] * len(X_test)

# PCA
pca = PCA(n_components=2)
coords = pca.fit_transform(X_combined)
var_exp = pca.explained_variance_ratio_ * 100

# Create DataFrame for plotting
pca_df = pd.DataFrame({
    'PC1': coords[:, 0],
    'PC2': coords[:, 1],
    'Status': ['Normozoospermia' if y == 0 else 'Oligozoospermia' for y in y_combined],
    'Tissue': tissue_labels
})

# Plot
fig, ax = plt.subplots(figsize=(10, 8))

color_map = {
    ('Normozoospermia', 'Blood'): '#e74c3c',
    ('Normozoospermia', 'Sperm'): '#2ecc71',
    ('Oligozoospermia', 'Blood'): '#e67e22',
    ('Oligozoospermia', 'Sperm'): '#3498db'
}

marker_map = {'Blood': 'o', 'Sperm': 's'}

for status in ['Normozoospermia', 'Oligozoospermia']:
    for tissue in ['Blood', 'Sperm']:
        mask = (pca_df['Status'] == status) & (pca_df['Tissue'] == tissue)
        if mask.any():
            ax.scatter(
                pca_df.loc[mask, 'PC1'],
                pca_df.loc[mask, 'PC2'],
                c=color_map[(status, tissue)],
                marker=marker_map[tissue],
                s=100, alpha=0.7,
                label=f"{status[:5]} ({tissue})",
                edgecolors='white'
            )

ax.set_xlabel(f'PC1 ({var_exp[0]:.1f}%)')
ax.set_ylabel(f'PC2 ({var_exp[1]:.1f}%)')
ax.set_title('PCA: Class Separation Analysis')
ax.legend(loc='best')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 9. Save Results

Save the analysis results to files.

In [ ]:
# Create output directory
OUTPUT_DIR = PROJECT_ROOT / "results" / "notebook_output"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Save performance summary
results_df.to_csv(OUTPUT_DIR / "model_performance.csv", index=False)
print(f"Saved: {OUTPUT_DIR / 'model_performance.csv'}")

# Save predictions
predictions_df = test_df[['sample_name', 'status', 'target']].copy()
for name, metrics in results.items():
    predictions_df[f'{name}_pred'] = metrics['y_pred']
    predictions_df[f'{name}_prob'] = metrics['y_prob']

predictions_df.to_csv(OUTPUT_DIR / "predictions.csv", index=False)
print(f"Saved: {OUTPUT_DIR / 'predictions.csv'}")

## 10. Custom Analysis

Modify the parameters below to run your own analysis.

In [ ]:
# ===========================================
# CUSTOMIZE YOUR ANALYSIS HERE
# ===========================================

# Change correlation threshold for feature selection
CORRELATION_THRESHOLD = 0.75  # Try 0.6, 0.7, 0.8, etc.

# Change the probe set
# PROBE_SET = "Controls"  # Uncomment to use control probes

# Change model parameters
# PLS_COMPONENTS = 3  # Try different number of PLS components

print(f"Current settings:")
print(f"  Correlation threshold: {CORRELATION_THRESHOLD}")
print(f"  Probe set: {PROBE_SET}")

## Next Steps

1. **Try different probe sets**: Change `PROBE_SET` to "Controls" or "EPIC_full"
2. **Adjust correlation threshold**: Modify `CORRELATION_THRESHOLD` to see how it affects results
3. **Add external validation**: Load GSE51245 data and validate predictions
4. **Run DMR analysis**: Use the R script for differentially methylated region analysis

For more details, see the project README.md and classifier/README.md files.